# Obtenção dos Dados:

Este projeto foi inspirado pelas atividades previamente realizadas por [Kizzy Terra](#https://github.com/kterra) e os dados utilizados estão neste [repositório](#https://github.com/kterra/ciencia-de-dados-aplicada), que segundo a própria autora foram dados obtidos pelo PRODES.

_"O projeto PRODES realiza o monitoramento por satélites do desmatamento por corte raso na Amazônia Legal e produz, desde 1988, as taxas anuais de desmatamento na região, que são usadas pelo governo brasileiro para o estabelecimento de políticas públicas."_ - [Coordenação Geral de Observação da Terra INPE](#http://www.obt.inpe.br/OBT/assuntos/programas/amazonia/prodes#:~:text=O%20projeto%20PRODES%20realiza%20o,o%20estabelecimento%20de%20pol%C3%ADticas%20p%C3%BAblicas.)

In [111]:
from modules import *



# DATASETS = [pd.read_csv(f'../Data/DesmatamentoMunicipios20{digits}.txt', sep=',', encoding='iso-8859-1') for digits in range(0,18)]
DATASETS = []

for i in range(0,19):
    a = '../Data/2000_2018/DesmatamentoMunicipiosAM2000.txt'
    if i < 10: DATASETS.append(pd.read_csv(f'../Data/2000_2018/DesmatamentoMunicipiosAM200{i}.txt', sep=',', encoding='iso-8859-1'))
    elif i>=10: DATASETS.append(pd.read_csv(f'../Data/2000_2018/DesmatamentoMunicipiosAM20{i}.txt', sep=',', encoding='iso-8859-1'))
     

type(DATASETS[0])


pandas.core.frame.DataFrame

**DATASETS** é uma lista com 19 dataframes com as informações dos arquivos que estão no diretório _`../Data/2000_2018`_

In [112]:
DATASETS[0].sample(3)

,Nr,Lat,Long,Latgms,Longms,Municipio,CodIbge,Estado,AreaKm2,Desmatado2000,Incremento19992000,Floresta2000,Nuvem2000,NaoObservado2000,NaoFloresta2000,Hidrografia2000,Soma
29,30,-3.37471,-66.45025,S 3 22 28.96,O 66 27 0.90,Juruá,1302207,AM,19849,114.9,NaN,18849.9,0.0,8.9,142.1,733.2,100
9,10,-4.73714,-69.52392,S 4 44 13.70,O 69 31 26.11,São Paulo de Olivença,1303908,AM,20467,103.8,NaN,19688.8,0.0,0.3,86.3,587.8,100
44,45,-3.74900,-60.14701,S 3 44 56.40,O 60 8 49.24,Careiro,1301100,AM,6127,964.4,NaN,4758.5,0.0,6.5,80.7,316.9,99


In [113]:
DATASETS[1].sample(2)

,Nr,Lat,Long,Latgms,Longms,Municipio,CodIbge,Estado,AreaKm2,Desmatado2001,Incremento20002001,Floresta2001,Nuvem2001,NaoObservado2001,NaoFloresta2001,Hidrografia2001,Soma
14,15,-0.65847,-60.17004,S 0 39 30.49,O 60 10 12.14,Presidente Figueiredo,1303536,AM,25568,583.4,58.5,21556.3,0.0,5.4,1.1,3421.8,99
57,58,-7.59202,-59.44152,S 7 35 31.27,O 59 26 29.47,Apuí,1300144,AM,54490,1013.0,57.2,52632.6,0.0,15.2,625.0,204.2,100


**MunicipalAdjascencyDict** é um dicionário contendo como chave o nome de uma cidade e como valor os municípios de adjascência da mesma

A referência de Ajascência municipal segue o modelo de mapa proposto pelo [Mapa do Escolar feito pelo IBGE](#https://geoftp.ibge.gov.br/produtos_educacionais/mapas_tematicos/mapas_do_brasil/mapas_estaduais/politico/amazonas.jpg) modificado em 2016, segue o exemplo:


In [114]:
MunicipalAdjascencyDict = {
'Urucurituba' : 
['Boa Vista do Ramos', 'Barreirinha', 'Itapiranga', 'Itacoatiara', 'Parintins', 'Urucara', 'Silves', 'Nhamunda'],

'Urucara' : 
['Nhamunda', 'Urucurituba', 'Itapiranga', 'Sao Sebastiao do Uatuma', 'Presidente Figueiredo'],

'Uarini' : 
['Manaa','Fonte Boa','Jurua', 'Alvaraes','Tefe'],

'Tonantins' : 
['Japura', 'Santo Antonio do Iça', 'Fonte Boa', 'Jutai'],

'Tefe' : 
['Tapaua', 'Manaa', 'Carauari', 'Jurua', 'Alvaraes', 'Uarini'],

'Tapaua' : 
['Itamarati', 'Pauini', 'Labrea', 'Canutama', 'Carauari', 'Tefe', 'Coari', 'Anori', 'Beruri', 'Manicore', 'Humaita'],

'Tabatinga' : 
['Benjamin Constant', 'Santo Antonio do Iça', 'Sao Paulo de Olivença'],

'Silves' : 
[ 'Urucurituba', 'Itacoatiara', 'Itapiranga'],

'Sao Sebastiao do Uatuma' : 
['Urucura', 'Presidente Figueiredo', 'Itapiranga'],

'Sao Paulo de Olivenca' : 
['Benjamin Constant', 'Jutai', 'Tabatinga', 'Santo Antonio do Iça', 'Amatur'],

'Sao Gabriel de Cachoeira' : 
['Japura', 'Santa Isabel do Rio Negro'],

'Santo Antonio do Ica' : 
['Sao Paulo de Olivença', 'Tabatinga', 'Amatur' , 'Tonantins', 'Jutai', 'Japura'],

'Santa Isabel do Rio Negro' : 
['Sao Gabriel da Cachoeira', 'Japura', 'Manaa', 'Barcelos'],

'Rio Preto da Eva' : 
['Itacoatiara', 'Manaus', 'Itapiranga', 'Presidente Figueiredo'],

'Presidente Figueiredo' : 
['Rio Preto de Eva', 'Manaus', 'Novo Airao', 'Itapiranga', 'Sao Sebastiao do Uatuma', 'Urucua'],

'Pauini' : #15
['Boca do Acre', 'Envira', 'Itamarati', 'Eirunepe','Tapaua', 'Labrea'],

'Parintins' : #16
['Nhamunda', 'Urucurituba', 'Barreirinha'],

'Novo Aripuana' : 
['Manicore', 'Apui', 'Borba'],

'Novo Airao' :  #18
['Barcelos', 'Codajas', 'Caapiranga', 'Manacapuru', 'Manaus', 'Presidente Figueiredo'],

'Nova Olinda do Norte' : 
['Borba', 'Maues', 'Autazes', 'Itacoatiara'],

'Nhamunda' :  
['Urucura', 'Urucurituba', 'Parintins', 'Maues'],

'Maues' : #21
['Apui', 'Borba', 'Itacoatiara', 'Boa Vista do Ramos', 'Barreirinha', 'Nova Olinda do Norte'],

'Manaa' : 
['Fonte boa', 'Japura', 'Santa Izabel do Rio Negro', 'Barcelos', 'Codajas', 'Coari', 'Tefe', 'Uarini'],

'Manicore' : 
['Humaita', 'Tapaua', 'Beruri', 'Borba','Novo Aripuana'],

'Manaus' : 
['Manacapuru','Itacoatiara','Careiro', 'Careiro da Varzea', 'Iranduba', 'Manacapuru', 'Novo Airao', 'Presidente Figueiredo', 'Rio Preto da Eva'],

'Manaquiri' : 
['Manacapuru', 'Borba', 'Beruri', 'Iranduba', 'Manaus', 'Careiro'],

'Manacapuru' : 
['Novo Airao', 'Caapiranga', 'Anama', 'Anori', 'Beruri', 'Manaquiri', 'Iranduba', 'Manaus'],

'Labrea' : 
['Boca do Acre', 'Pauini', 'Itamarati', 'Tapaua', 'Canutama'],

'Jutai' :  
['Itamarati', 'Eirunepe', 'Benjamin Constant', 'Sao Paulo de Olivença','Amatur','Santo Antonio do Iça', 'Tonantins', 'Fonte Boa', 'Jurua', 'Carauari'],

'Jurua' :  
['Carauari', 'Jutai', 'Fonte Boa', 'Uarini', 'Alvaraes', 'Tefe'],

'Japura' : 
['Santo Antonio de Iça',' Tonantins','Fonte Boa', 'Manaa','Santa Isabel do Rio Negro',' Sao Gabriel de Cachoeira'],

'Itapiranga' :  
['Presidente Figueiredo','Sao Sebastiao do Uatuma','Urucura','Urucurituba', 'Silves', 'Rio Preto de Eva', 'Itacoatiara'],

'Itamarati' : 
['Envira', 'Eirunepe', 'Jutai', 'Carauari', 'Tapua','Labrea','Pauini'],

'Itacoatiara' : 
['Autazes', 'Careiro da Varzea', 'Maues', 'Rio Preto da Eva', 'Nova Olinda do Norte', 'Manaus', 'Silves', 'Itapiranga', 'Urucurituba', 'Boa Vista do Ramos'],

'Iranduba' :  
['Novo Airao','Careiro', 'Careiro da Varzea', 'Manacapuru', 'Manaquiri', 'Manaus'],

'Ipixuna' : 
['Eirunepe', 'Benjamin Constant', 'Atalaia do Norte', 'Guajara'],

'Humaita' : 
['Canutama', 'Tapaua', 'Manicore'],

'Guajara' : 
['Atalaia do Norte', 'Ipixuna'],

'Fonte Boa' : 
['Jutai', 'Jurua', 'Tonantins', 'Japura', 'Manaa', 'Uarini'],

'Envira' : 
['Pauini', 'Itamarati', 'Eirunepe'],

'Eirunepe' : 
['Benjamin Constant', 'Ipixuna', 'Envira', 'Itamarati', 'Jutai','Pauini'],

'Codajas' : 
['Anama','Anori','Coari','Manaa','Barcelos','Novo Airao','Caapiranga'],

'Coari' : 
['Manaa', 'Codajas', 'Tefe', 'Tapua', 'Anori','Barcelos'],

'Careiro da Varzea' : 
['Iranduba', 'Careiro', 'Manaus', 'Itacoatiara', 'Autazes'],

'Careiro' : 
['Iranduba', 'Manacapuru', 'Manaquiri', 'Borba', 'Autazes', 'Careiro da Varzea', 'Manaus'],

'Carauari' : 
['Tefe', 'Alvaraes', 'Jurua', 'Jutai', 'Itamarati', 'Tapua'],

'Canutama' :  
['Labrea', 'Tapaua', 'Humaita'],

'Caapiranga' : 
['Anama', 'Manacapuru', 'Novo Airao', 'Codajas'],

'Borba' : 
['Maues', 'Nova Olinda do Norte', 'Autazes', 'Careiro', 'Manaquiri', 'Beruri', 'Manicore', 'Novo Aripuana', 'Apui'],

'Boca do Acre' : 
['Pauini', 'Labrea'],

'Boa Vista do Ramos' : 
['Maues','Barreirinha', 'Urucurituba', 'Itacoatiara'],

'Beruri' : 
['Manicore', 'Tapua', 'Anori', 'Anama', 'Manacapuru', 'Manaquiri'],

'Benjamin Constant' : 
['Atalaia do Norte', 'Ipixuna', 'Eirunepe', 'Jutai', 'Sao Paulo de Olivença', 'Tabatinga'],

'Barreirinha' : 
['Maues', 'Boa Vista do Ramos', 'Urucurituba', 'Parintins'],

'Barcelos' : 
['Santa Isabel do Rio Negro', 'Novo Airao', 'Manaa', 'Codajas', 'Coari'],

'Autazes' : 
['Manaus', 'Careiro', 'Careiro da Varzea', 'Borba', 'Itacoatiara', 'Nova Olinda do Norte'],

'Atalaia do Norte' :  
['Benjamim Constant', 'Guajara','Ipixuna'],

'Apui' : 
['Novo Aripuana', 'Borba', 'Maues'],

'Anori' : 
['Beruri', 'Tapua', 'Coari', 'Codajas', 'Anama'],

'Anama' :  
['Manacapuru', 'Beruri',' Anori','Codajas','Caapiranga'],

'Amatur' : 
['Sao Paulo de OLivença', 'Jutai', 'Santo Antonio do Iça'],

'Alvaraes' : 
['Tefe', 'Carauari', 'Jurua', 'Uarini'],
}

# Limpeza e Inserção de Dados:

A remoção de caracteres e de colunas não utilizadas facilitará o trabalho com as informações essênciais. Além disso a normalização dos dados e estabelecimento de adjascência entre municípios será essêncial para análise destes e modelagem dos grafos.

### Removendo colunas irrelevantes:

In [115]:
print(DATASETS[0].columns)

Index(['Nr', 'Lat', 'Long', 'Latgms', 'Longms', 'Municipio', 'CodIbge',
       'Estado', 'AreaKm2', 'Desmatado2000', 'Incremento19992000',
       'Floresta2000', 'Nuvem2000', 'NaoObservado2000', 'NaoFloresta2000',
       'Hidrografia2000', 'Soma '],
      dtype='object')


In [116]:
DATASETS[0].drop(columns = ['Incremento19992000'], inplace = True)

In [117]:

DATASETS[0]

for index, value in enumerate(DATASETS):
    if index < 10:
        value.drop(columns = ['Nr'], inplace = True)
        value.drop(columns = ['CodIbge'], inplace = True)
        value.drop(columns = [f'Nuvem200{index}'], inplace = True)
        value.drop(columns = ['Estado'], inplace = True)
        value.drop(columns = ['Soma '], inplace = True)
        value.drop(columns = ['Latgms'], inplace = True)
        value.drop(columns = ['Longms'], inplace = True)
        
    else:
        value.drop(columns = ['Nr'], inplace = True)
        value.drop(columns = ['CodIbge'], inplace = True)
        value.drop(columns = [f'Nuvem20{index}'], inplace = True)
        value.drop(columns = ['Estado'], inplace = True)
        value.drop(columns = ['Latgms'], inplace = True)
        value.drop(columns = ['Longms'], inplace = True)

DATASETS[0].sample(2)

,Lat,Long,Municipio,AreaKm2,Desmatado2000,Floresta2000,NaoObservado2000,NaoFloresta2000,Hidrografia2000
39,-7.61678,-70.19515,Envira,7793,138.8,7586.0,5.3,0.1,62.8
17,-6.83379,-60.23604,Novo Aripuanã,41452,534.7,38526.6,27.9,1620.2,742.6


### Tirando acento dos caracteres

In [118]:
#Criando método para retirar os acentos das palavras dentro deste dataset:
def substituir_caracteres(columns):  
  columns = columns.replace('Á','A').replace('Â','A').replace('Ã','A').replace('À','A').replace('É','E').replace('Ê','E').replace('È','E').replace('Í','I').replace('Ì','I').replace('Î','I').replace('Ó','O').replace('Ò','O').replace('Õ','O').replace('Ô','O').replace('Ú','U').replace('Ù','U').replace('Û','U').replace('Ç','C').replace('á','a').replace('â','a').replace('ã','a').replace('à','a').replace('é','e').replace('ê','e').replace('è','e').replace('í','i').replace('ì','i').replace('î','i').replace('ó','o').replace('ò','o').replace('õ','o').replace('ô','o').replace('ú','u').replace('ù','u').replace('û','u').replace('ç','c').replace('-', ' ').replace('.','')
  return columns

for index,dataset in enumerate(DATASETS):
  DATASETS[index]['Municipio'] = DATASETS[index]['Municipio'].apply(substituir_caracteres)

DATASETS[0]['Municipio'].sample(15)

49             Boca do Acre
48                    Borba
61                 Alvaraes
41                  Codajas
26               Manacapuru
24                   Manaus
31               Itapiranga
2                    Uarini
54                 Barcelos
29                    Jurua
56         Atalaia do Norte
52        Benjamin Constant
14    Presidente Figueiredo
51                   Beruri
15                   Pauini
Name: Municipio, dtype: object

### Criando coluna de fronteira entre municípios para todos os datasets

In [119]:

MunicipalAdjascencyList = [x for x in MunicipalAdjascencyDict.values()]

for i in DATASETS:
    i['Fronteiras'] = MunicipalAdjascencyList

DATASETS[0].sample(4)

,Lat,Long,Municipio,AreaKm2,Desmatado2000,Floresta2000,NaoObservado2000,NaoFloresta2000,Hidrografia2000,Fronteiras
46,-7.45596,-63.90278,Canutama,30278,451.2,28536.2,5.5,988.3,296.8,"[Labrea, Tapaua, Humaita]"
2,-3.20648,-65.43058,Uarini,10448,194.2,9721.7,0.0,100.3,431.8,"[Manaa, Fonte Boa, Jurua, Alvaraes, Tefe]"
11,-3.08079,-69.06723,Santo Antonio do Ica,12737,111.3,12018.8,65.2,90.4,451.3,"[Sao Paulo de Olivença, Tabatinga, Amatur, Ton..."
61,-3.67425,-65.34586,Alvaraes,6027,258.3,5603.1,0.0,38.0,127.6,"[Tefe, Carauari, Jurua, Uarini]"


### Estabelecendo média de desmatamento:

In [120]:
# mean_column = []

mean = DATASETS[0]['Desmatado2000'].mean()

# print(mean)
for index, dataframe in enumerate(DATASETS):
    mean_column = []
    if index < 10:
        mean = dataframe[f'Desmatado200{index}'].mean()
        for i in dataframe[f'Desmatado200{index}']:
            mean_column.append(True) if i > mean else mean_column.append(False)
    else:
        mean = dataframe[f'Desmatado20{index}'].mean()
        for i in dataframe[f'Desmatado20{index}']:
            mean_column.append(True) if i > mean else mean_column.append(False)

    DATASETS[index]['MaiorQueAMédia'] = mean_column

# exit()

        
#    if DATASETS[0]['Desmatado2000'].mean() < 

# Exportando dados em csv para análise:

A análise dos dado junto à modelagem do grafo são realizados em um arquivo próprio, sendo este exclusivamente utilizado para limpeza dos dados. Os arquivos com dados tratados estão na pasta CleanData2000_2018.

In [121]:

for index, dataframe in enumerate(DATASETS):
        path_to_save = r'~/Área de trabalho/ALL/Desktop/CEFET2/AEDS2/Atividades e Trabalhos/Desenvolvimento/DataAnalysis/DesmatamentoNoAmazonas/Data/CleanData2000_2018'
        if index < 10:
                dataframe.to_csv(path_to_save+f'Cleaned200{index}.csv' , index = False, encoding='utf-8')
        else:
                dataframe.to_csv(path_to_save+f'Cleaned20{index}.csv', index = False, encoding='utf-8')